In [16]:
import datetime

import pandas as pd
import numpy as np
import astropy.time

import pint
ureg = pint.UnitRegistry()


Q_ = ureg.Quantity

In [12]:
def date2six(date):
    """compute the phases of six tidal components at a date"""
    t = pd.to_datetime(date)
    mjdate =  t.to_julian_date() - 2400000.5

    # astrol from here
    TIME = mjdate

    # this is 2000-01-01T12:00 in ET/TDT/TDB....
    # https://en.wikipedia.org/wiki/Ephemeris_time
    T    = TIME - 51544.4993   # reference to 2000/1/1 1200 o'clock

    # 
    #      --- perform translations using translation table of symbols:
    # 
    #          nr Cartwright  Doodson  Brown
    #          1  tau         tau      360*t-D+180
    #          2  q           s        L
    #          3  q'          h        L'
    #          4  p           p        \overline(\omega)
    #          5  N           -N'      \Omega
    #          6  p'          p_1      \overline{\omega}'
    # 

    # instantiate 
    six = [None for i in range(6)]
    six[1] = 218.3164 + 13.17639648 * T
    six[2] = 280.4661 +  0.98564736 * T
    six[3] =  83.3535 +  0.11140353 * T
    six[4] = 125.0445 -  0.05295377 * T
    six[5] = 282.9384 +  0.0000471  * T


    # 
    #      --- get them in the right quadranthttp://localhost:8889/notebooks/Untitled.ipynb?kernel_name=python3#
    # 

    circle = 360
    for i in range(1, 6):
        six[i] =  np.mod(six[i], circle)
        if (six[i] < 0):
            six[i] = six[i] + circle

    # 
    #          argument 1 in the doodson number denotes the mean lunar time.
    #          According to equation 13 and the inline remark after equation 14
    #          it is computed by
    #          alpha_G = 360 * "Universal time in fractions of a day" + q'(T) - 180
    #          tau = alpha_G - q
    # 

    UT     = mjdate - int(mjdate)
    six[0] = 360 * UT + six[2] - 180 - six[1]
    return six

In [13]:
# phases at 2000-01-01
phases = date2six(datetime.datetime(2000, 1, 1))

# phases in 360 degrees 
phases

[-111.76345896434583,
 211.73742523749473,
 279.9739662731489,
 83.29787621747064,
 125.07093981736116,
 282.93837648297]

In [25]:
periods = [347.8092509724891, 13.17639648, 0.98564736, 0.11140353, 0.05295377, 0.0000471]
epochs  = []
for i in range(0, 6):
    circle = Q_(360, 'deg')
    phase = Q_(phases[i], 'deg')
    # deg / day
    period = Q_(periods[i], 'deg/day')
    day_in_year = Q_(365.2425, 'days/year')
    days_since_2000  = phase / period 
    years_since_2000 = days_since_2000 / day_in_year
    epoch = astropy.time.Time('2000-1-1')
    delta_t = astropy.time.TimeDelta(days_since_2000.magnitude, format='jd')
    epochs.append(epoch + delta_t)


In [34]:
names_cartwright  = ["$\tau$", "$q$", "$q'$", "$p$", "$N$", "$p'$"]
names_doodson = ["$\tau$", "$s$", "$h$", "$p$", "$-N'$", "$p_1$"]

data = {
    "phase": phases,
    "period": periods,
    "epoch": epochs,
    "name_cartwright": names_cartwright,
    "name_doodson": names_doodson
}
df = pd.DataFrame(data)


In [35]:
df.to_json('tidal-components.json', default_handler=str)

In [36]:
df

,phase,period,epoch,name_cartwright,name_doodson
0,-111.763459,347.809251,1999-12-31 16:17:16.613,$\tau$,$\tau$
1,211.737425,13.176396,2000-01-17 01:40:00.354,$q$,$s$
2,279.973966,0.985647,2000-10-11 01:13:13.078,$q'$,$h$
3,83.297876,0.111404,2002-01-17 17:06:49.863,$p$,$p$
4,125.070940,0.052954,2006-06-19 21:20:29.723,$N$,$-N'$
5,282.938376,0.000047,18447-02-10 05:08:11.149,$p'$,$p_1$
